# LSTM

RNN 的一个问题是当sequence 特别长的时候，因为loss 在向回传递逐层减弱，所以有时会无法扑捉到token 之间的关系，考虑下面两句话：

- The young woman went to the movies with her friends.

- The young woman, having found a free ticket on the ground, went to the movies.

第一句话因为主语和谓语紧挨着，所以很容易捕捉这类关系。第二句话，主语和谓语之间加入了一个从句，所以很有可能无法捕捉主语和谓语的关系。

⚠️ 没有捕捉到这个关系的影响是什么？

LSTM 解决这个问题的方法是加入了一个`state` 的concept， 这个state 可以看作是memory。memory 的作用是，通过training，可以学习到what to remember, 同时，网络其余的部分学习如何利用remember的和输入的数据来做预测。

通过memory，可以捕获到更长的依赖关系。

⚠️ hard to think.

使用LSTM，除了可以predict，还可以generate text.

LSTM 网络图如下：

<img src="img/lstm.png" alt="drawing" width="500"/>

可以看出，这是一个RNN unrolled version + memory state.

下面，我们来看每个LSTM Layer。






### LSTM Layer

#### 输入

- input instance of current time step
    - 300-element vector
- output from previous time step
    - 50-element vector
- concatenation: 把两个input vectors 拼接成一个长度为350-element vector.

<img src="img/lstm_layer_input.png" alt="drawing" width="700"/>

input 一共经过3个gates，每个gate 都是一个feed forward network layer, gate 的weights 决定了有多少信息可以go through to the cell's state (memory).

- forget gate
- input/candidate gate (2 branches)
- update/output gate

参数个数分析：
- 每个gate 的每个neuron 连接为长度为350 的vector + 1个bias， 总共351 个weights。
- 每个gate有50个neurons，总共为351 * 50 = 17550
- 一共3个gates, candidate gate 有两个分支，参数个数一样，总共可以看作4个gates，参数个数为：17750 * 4 = 70200
- output layer，LSTM 的输出是400 * 50 (每个step 输出长度为50的"thought vector"，一共50 steps), flatten 之后长度20000，加一个bias 一共20001
- 总共 70200 + 20001 = 90201 个参数

<img src="img/forget_gate.png" alt="drawing" width="500"/>

注意，对于第一个token，step t-1 的50-element vector 补零。

### 1. Forget Gate

the goal is to learn how much of the cell's memory you want to erase. The idea behind wanting to forget is as important as wanting to remember.

forget gate 本身是一个feed forward network:
- n neurons
- m + n + 1 weights for each neuron (300 + 50 + 1)
- activation function: sigmoid
- output: 0 ~ 1

<img src="img/forget_gate_weights.png" alt="drawing" width="700"/>

forget gate 的输出类似于一个mask, 值接近1 代表通过率高，即保留记忆；值接近0 代表通过率低，即删除记忆。然后这个"mask" 和memory vector 做element-wise 乘法，更新memory，过程如下图所示。这就是forget gate 怎么做到forget things 的。forget 是指，更新memory，使某些维度的信息量减少。

<img src="img/forget_gate_calculation.png" alt="drawing" width="500"/>

### 2. Candidate gate

goal: how much to augment the memory based on:
- concatenated input
    - input of step t
    - output of step t-1
    
如下图所示，candidate gate 包含 2 个 branches:
1. decide which input vector elements are worth remembering
    - 类似于forget gate, sigmoid function, 输出 0 ～ 1
2. Rout the remembered input elements to the right memory slot.
    - what value you are going to update the memory with?
    - activation: tanh
    - -1 ~ 1

<img src="img/candidate_gate.png" alt="drawing" width="500"/>

Output:
- 然后我们把两个vector 做element-wise multiplication. 

最后，这个output 和之前的updated memory 做element-wise addition，实现remember new things.


### 3. Output/Update gate

flow 1 (gate): 
- Input_1: concatenated input
- n neurons
- activation function: sigmoid
- output_1: n-dimensional output between 0 and 1

flow 2 (mask):
- input_2: updated memory vector
- tanh function applied elementwise
- output_2: n-dimensional vector （value between -1 and 1）

注意，这里直接使用tanh function，并没有neuron（即没有weight），所以可以称为mask，但不能称为gate。

然后，output_1 element-wise multiplication with output_2.
- 生成一个新的n-dimensional vector (step official output)
    - 传到step t+1
    - layer's output

整个过程如下图所示。

<img src="img/update_gate.png" alt="drawing" width="700"/>

⚠️ 图上又个bug
